In [1]:
# move this down
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel 
ds = pd.read_csv('scotch_review.csv')

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pickle
import csv
import time

In [3]:
# Finds list of whiskey names

URL1 = 'https://scotchnoob.com/reviews/?pagenum=1&dateFrom=11/10/2010&dateTo=04/11/2021&shownum=-1&orderBy=title&order=ASC&priceFrom=0&priceTo=-1&region=All&rating=All&tag_peated=false&tag_sherried=false&tag_indie=false'

page1 = requests.get(URL1)

soup1 = BeautifulSoup(page1.text, 'lxml')


In [105]:
# Finds whiskey names
whiskey_names = []

name0 = soup1.find('div', {'id' : 'filter_results'})

whiskey_names = name0.find_all('h4')


for w in range(len(whiskey_names)):

    whiskey_names[w] = whiskey_names[w].text




# Finds whiskey urls

whiskey_urls = []

for tag in name0.find_all('a', href = True):
   
    if (str(tag['href']) == '/?page_id=264') : continue
    whiskey_urls.append(str(tag['href']))



# Finds description of whiskey

whiskey_desc = []


for r in range(len(whiskey_names)):
    
    if (whiskey_files[r] == 'TheBenRiach(19year)Peated/OlorosoFinish1994'): 
        page0 = pickle.load(open('pages/TheBenRiach(19year)Peated-OlorosoFinish1994.p', 'rb'))
    
    else:
        page0 = pickle.load(open('pages/' + whiskey_files[r] + '.p', 'rb'))
    
    
    soup0 = BeautifulSoup(page0.text, 'lxml')
 
    
    # Builds temp array to join and append to whiskey_desc array
    
    class0 = soup0.find('div', {'class' : 'entry-content'})
    class1 = class0.find_all('strong')
    
   
    whiskey_desc_mass = []
    desc_start_check = False
    
    for e in range(len(class1)):
         
        if(class1[e].text == 'Nose' or desc_start_check == True):
        
            desc_start_check = True
     
            whiskey_desc_mass.append(class1[e].next_sibling)
                
        else:
            continue
    
        
    [''.join(whiskey_desc_mass) for w in whiskey_desc_mass]
    
    
    whiskey_desc.append(whiskey_desc_mass)
    


In [175]:
# Don't run this again!!
# Don't delete, for posssible referance

whiskey_files = []
for r in range(len(whiskey_names)):
    
    URL0 = whiskey_urls[r]
    
    page0 = requests.get(URL0)
    
    
    whiskey_files.append(str(whiskey_names[r]).replace(' ',''))
    

    if (whiskey_files[r] == 'TheBenRiach(19year)Peated/OlorosoFinish1994'): 
        pickle.dump(page0, open('pages/TheBenRiach(19year)Peated-OlorosoFinish1994.p', 'wb'))
    
    else:
        pickle.dump(page0, open('pages/' + whiskey_files[r] + '.p', 'wb'))